In [2]:
import numpy as np

In [3]:
pwd

'/content'

In [4]:
import dlib

In [5]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive')

Mounted at /content/drive


In [22]:
import os
import numpy as np
import pandas as pd
import cv2 as cv2
import random as random
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import datasets, layers, models
from tensorflow import keras

In [8]:
!unzip Train.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/WIN_20180926_16_46_40_Pro_Left_Swipe_new/WIN_20180926_16_46_40_Pro_00046.png  
  inflating: train/WIN_20180926_16_46_40_Pro_Left_Swipe_new/WIN_20180926_16_46_40_Pro_00048.png  
  inflating: train/WIN_20180926_16_46_40_Pro_Left_Swipe_new/WIN_20180926_16_46_40_Pro_00050.png  
  inflating: train/WIN_20180926_16_46_40_Pro_Left_Swipe_new/WIN_20180926_16_46_40_Pro_00052.png  
  inflating: train/WIN_20180926_16_46_40_Pro_Left_Swipe_new/WIN_20180926_16_46_40_Pro_00054.png  
  inflating: train/WIN_20180926_16_46_40_Pro_Left_Swipe_new/WIN_20180926_16_46_40_Pro_00056.png  
  inflating: train/WIN_20180926_16_46_40_Pro_Left_Swipe_new/WIN_20180926_16_46_40_Pro_00058.png  
  inflating: train/WIN_20180926_16_46_40_Pro_Left_Swipe_new/WIN_20180926_16_46_40_Pro_00060.png  
  inflating: train/WIN_20180926_16_46_40_Pro_Left_Swipe_new/WIN_20180926_16_46_40_Pro_00062.png  
   creating: train/WIN_20180926_16_47_00_Pro_Right_Swipe_new/
  inf

In [9]:
from keras.applications.vgg16 import VGG16

In [10]:
# We need to get all the paths for the images to later load them
imagepaths = []

# Go through all the files and subdirectories inside a folder and save path to images inside list
for root, dirs, files in os.walk(".", topdown=False): 
  for name in files:
    path = os.path.join(root, name)
    if path.endswith("png"): # We want only the images
      imagepaths.append(path)

print(len(imagepaths)) # If > 0, then a PNG image was loaded

12090


In [11]:
print(imagepaths[0])

./train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00012.png


In [12]:
X = [] # Image data
y = [] # Labels

# Loops through imagepaths to load images and labels into arrays
for path in imagepaths:
  img = cv2.imread(path) # Reads image and returns np.array
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Converts into the corret colorspace (GRAY)
  img = cv2.resize(img, (256, 256)) # Reduce image size so training can be faster
  X.append(img)
  
  # Processing label in image path
  category = path.split("/")[2]
  label1 = category.split("_")[6] # We need to convert 10_down to 00_down, or else it crashes
  if label1 == "Right Swipe" or label1 == "Right":
    y.append(0)
  elif label1 == "Left Swipe" or label1 == "Left":
    y.append(1)
  else:
    y.append(2)

# Turn X and y into np.array to speed up train_test_split
X = np.array(X, dtype="uint8")
X = X.reshape(len(imagepaths), 256, 256, 1) # Needed to reshape so CNN knows it's different images
y = np.array(y)

print("Images loaded: ", len(X))
print("Labels loaded: ", len(y))

print(y[0], imagepaths[0]) # Debugging#

Images loaded:  12090
Labels loaded:  12090
0 ./train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00012.png


In [13]:
from sklearn.model_selection import train_test_split # Helps with organizing data for training
from sklearn.metrics import confusion_matrix # Helps present results as a confusion-matrix
ts = 0.3 # Percentage of images that we want to use for testing. The rest is used for training.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ts, random_state=42)

In [14]:
model1 = models.Sequential()
model1.add(layers.Conv2D(32, (3,3), input_shape = (256, 256, 3), activation = 'relu'))
model1.add(layers.MaxPooling2D(pool_size = (2,2)))
model1.add(layers.Dropout(0.05))
model1.add(layers.BatchNormalization())
model1.add(layers.Conv2D(240, (3,3), activation = 'relu'))
model1.add(layers.MaxPooling2D(pool_size = (2,2)))
model1.add(layers.Dropout(0.07))
model1.add(tf.keras.layers.BatchNormalization())
model1.add(layers.Conv2D(240, (3,3), activation = 'relu'))
model1.add(layers.MaxPooling2D(pool_size=(2,2)))
model1.add(layers.Dropout(0.15))
model1.add(tf.keras.layers.BatchNormalization())
model1.add(layers.Flatten())
model1.add(layers.Dense(200,activation = 'relu'))
model1.add(layers.Dropout(0.4))
model1.add(layers.Dense(3, activation = 'softmax'))
model1.compile(loss = 'kullback_leibler_divergence',optimizer= 'adam',
              metrics = ['accuracy'])
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 127, 127, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 127, 127, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 240)     69360     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 240)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 62, 62, 240)       0

In [15]:
# model1.load_weights("modelCNN.h5")
from keras.models import Sequential
from keras.models import Model
model1 = Model(inputs=model1.input , outputs=model1.output)

In [16]:
model1.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [17]:
model1.fit(X_train, y_train, epochs=5,validation_data=(X_test, y_test))

Epoch 1/5


ValueError: ignored

In [18]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten

In [19]:
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(256, 256, 1))) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [20]:
model.compile(optimizer='adam', # Optimization routine, which tells the computer how to adjust the parameter values to minimize the loss function.
              loss='sparse_categorical_crossentropy', # Loss function, which tells us how bad our predictions are.
              metrics=['accuracy']) # List of metrics to be evaluated by the model during training and testing.

In [27]:
model = keras.models.load_model('handrecognition_model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 252, 252, 32)      832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 124, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 57600)             0

In [24]:
model.fit(X_train, y_train, epochs=5,validation_data=(X_test, y_test))

Epoch 1/5
265/265 [==============================] - 759s 3s/step - loss: 33.9721 - accuracy: 0.7352 - val_loss: 0.1233 - val_accuracy: 0.9620
Epoch 2/5
265/265 [==============================] - 764s 3s/step - loss: 0.0298 - accuracy: 0.9906 - val_loss: 0.0205 - val_accuracy: 0.9931
Epoch 3/5
265/265 [==============================] - 773s 3s/step - loss: 0.0370 - accuracy: 0.9886 - val_loss: 0.0271 - val_accuracy: 0.9928
Epoch 4/5
265/265 [==============================] - 774s 3s/step - loss: 0.0237 - accuracy: 0.9943 - val_loss: 0.0462 - val_accuracy: 0.9859
Epoch 5/5
265/265 [==============================] - 772s 3s/step - loss: 0.0283 - accuracy: 0.9951 - val_loss: 0.0258 - val_accuracy: 0.9912


In [26]:
model.save('handrecognition_model.h5')

In [28]:
classes = [
    "Swiping Right",
    "Swiping Left",
    "Zoom In"
    ]

In [30]:
import cv2

In [44]:
to_predict = []
num_frames = 0
cap = cv2.VideoCapture(0)
classe =''

while(True):
    # Capture frame-by-frame
  ret,frame = cap.read()

    # Our operations on the frame come here
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  to_predict.append(cv2.resize(gray, (256, 256)))
    
         
  if len(to_predict) == 30:
    frame_to_predict = np.array(to_predict, dtype=np.float32)
    frame_to_predict = normaliz_data(frame_to_predict)
        #print(frame_to_predict)
    predict = new_model.predict(frame_to_predict)
    classe = classes[np.argmax(predict)]
        
    print('Classe = ',classe, 'Precision = ', np.amax(predict)*100,'%')


        #print(frame_to_predict)
    to_predict = []
        #sleep(0.1) # Time in seconds
        #font = cv2.FONT_HERSHEY_SIMPLEX
  cv2.putText(frame, classe, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0),1,cv2.LINE_AA)


    # Display the resulting frame
  cv2.imshow('Hand Gesture Recognition',frame)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break
cap.release()
cv2.destroyAllWindows()

AttributeError: ignored